In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % save_interval == 0:
                    self.save_imgs(epoch,global_step)



    def save_imgs(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_dcgan'):
            os.mkdir('images_dcgan')
        fig.savefig("images_dcgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=30, batch_size=64, save_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.287789, acc.: 23.44%] [G loss: 0.541541]
epoch:0 step:2 [D loss: 0.801238, acc.: 50.78%] [G loss: 0.979042]
epoch:0 step:3 [D loss: 0.587529, acc.: 66.41%] [G loss: 1.276818]
epoch:0 step:4 [D loss: 0.698658, acc.: 64.84%] [G loss: 1.160223]
epoch:0 step:5 [D loss: 0.705121, acc.: 56.25%] [G loss: 0.972318]
epoch:0 step:6 [D loss: 0.636824, acc.: 64.06%] [G loss: 0.983631]
epoch:0 step:7 [D loss: 0.504648, acc.: 76.56%] [G loss: 1.083948]
epoch:0 step:8 [D loss: 0.516334, acc.: 78.12%] [G loss: 0.685956]
epoch:0 step:9 [D loss: 0.525487, acc.: 76.56%] [G loss: 0.833036]
epoch:0 step:10 [D loss: 0.739761, acc.: 59.38%] [G loss: 1.261822]
epoch:0 step:11 [D loss: 0.696613, acc.: 60.16%] [G loss: 1.636428]
epoch:0 step:12 [D loss: 0.648218, acc.: 68.75%] [G loss: 1.611941]
epoch:0 step:13 [D loss: 0.598673, acc.: 70.31%] [G loss: 1.678117]
epoch:0 step:14 [D loss: 0.625595, acc.: 67.19%] [G loss: 1.873719]
epoch:0 step:15 [D loss: 0.753085, acc.: 60.94%] [G loss:

epoch:0 step:126 [D loss: 0.766735, acc.: 51.56%] [G loss: 1.232647]
epoch:0 step:127 [D loss: 0.821657, acc.: 43.75%] [G loss: 1.175924]
epoch:0 step:128 [D loss: 0.724581, acc.: 55.47%] [G loss: 0.919375]
epoch:0 step:129 [D loss: 0.899751, acc.: 41.41%] [G loss: 1.123849]
epoch:0 step:130 [D loss: 0.760593, acc.: 47.66%] [G loss: 1.171886]
epoch:0 step:131 [D loss: 0.796568, acc.: 47.66%] [G loss: 1.165784]
epoch:0 step:132 [D loss: 0.894369, acc.: 42.97%] [G loss: 1.051179]
epoch:0 step:133 [D loss: 0.844079, acc.: 46.88%] [G loss: 1.044677]
epoch:0 step:134 [D loss: 0.868471, acc.: 46.88%] [G loss: 1.032505]
epoch:0 step:135 [D loss: 0.729335, acc.: 54.69%] [G loss: 1.134413]
epoch:0 step:136 [D loss: 0.807881, acc.: 50.00%] [G loss: 1.086956]
epoch:0 step:137 [D loss: 0.843468, acc.: 50.78%] [G loss: 0.986600]
epoch:0 step:138 [D loss: 0.719735, acc.: 58.59%] [G loss: 1.057983]
epoch:0 step:139 [D loss: 0.815333, acc.: 45.31%] [G loss: 1.192881]
epoch:0 step:140 [D loss: 0.880056

...

epoch:29 step:27606 [D loss: 0.442308, acc.: 83.59%] [G loss: 1.382840]
epoch:29 step:27607 [D loss: 0.542967, acc.: 67.97%] [G loss: 1.332847]
epoch:29 step:27608 [D loss: 0.403834, acc.: 88.28%] [G loss: 1.451610]
epoch:29 step:27609 [D loss: 0.573017, acc.: 71.88%] [G loss: 1.542407]
epoch:29 step:27610 [D loss: 0.805540, acc.: 50.78%] [G loss: 1.287800]
epoch:29 step:27611 [D loss: 0.788479, acc.: 47.66%] [G loss: 1.237721]
epoch:29 step:27612 [D loss: 0.557492, acc.: 70.31%] [G loss: 1.331928]
epoch:29 step:27613 [D loss: 0.638589, acc.: 62.50%] [G loss: 1.107356]
epoch:29 step:27614 [D loss: 0.597036, acc.: 66.41%] [G loss: 1.363875]
epoch:29 step:27615 [D loss: 0.725462, acc.: 57.03%] [G loss: 1.127663]
epoch:29 step:27616 [D loss: 0.350313, acc.: 90.62%] [G loss: 1.751356]
epoch:29 step:27617 [D loss: 0.598985, acc.: 63.28%] [G loss: 1.382268]
epoch:29 step:27618 [D loss: 0.781463, acc.: 49.22%] [G loss: 0.925449]
epoch:29 step:27619 [D loss: 0.718834, acc.: 60.16%] [G loss: 1.

epoch:29 step:27722 [D loss: 0.186591, acc.: 98.44%] [G loss: 1.722144]
epoch:29 step:27723 [D loss: 0.295684, acc.: 93.75%] [G loss: 1.894420]
epoch:29 step:27724 [D loss: 0.245623, acc.: 96.09%] [G loss: 1.833690]
epoch:29 step:27725 [D loss: 0.293265, acc.: 94.53%] [G loss: 1.739692]
epoch:29 step:27726 [D loss: 0.338001, acc.: 90.62%] [G loss: 1.712299]
epoch:29 step:27727 [D loss: 0.268901, acc.: 90.62%] [G loss: 1.399071]
epoch:29 step:27728 [D loss: 0.277509, acc.: 96.88%] [G loss: 1.945883]
epoch:29 step:27729 [D loss: 0.373709, acc.: 85.94%] [G loss: 1.652862]
epoch:29 step:27730 [D loss: 0.407971, acc.: 86.72%] [G loss: 1.736150]
epoch:29 step:27731 [D loss: 0.359976, acc.: 86.72%] [G loss: 2.166854]
epoch:29 step:27732 [D loss: 1.059564, acc.: 37.50%] [G loss: 1.291322]
epoch:29 step:27733 [D loss: 0.888270, acc.: 49.22%] [G loss: 1.250522]
epoch:29 step:27734 [D loss: 0.327416, acc.: 91.41%] [G loss: 1.576167]
epoch:29 step:27735 [D loss: 0.834096, acc.: 46.88%] [G loss: 1.

epoch:29 step:27836 [D loss: 0.161275, acc.: 98.44%] [G loss: 1.648754]
epoch:29 step:27837 [D loss: 0.151311, acc.: 98.44%] [G loss: 1.921358]
epoch:29 step:27838 [D loss: 0.156735, acc.: 98.44%] [G loss: 1.870427]
epoch:29 step:27839 [D loss: 0.235050, acc.: 96.88%] [G loss: 2.268838]
epoch:29 step:27840 [D loss: 0.351085, acc.: 86.72%] [G loss: 2.077252]
epoch:29 step:27841 [D loss: 0.520585, acc.: 76.56%] [G loss: 1.421867]
epoch:29 step:27842 [D loss: 0.309468, acc.: 92.97%] [G loss: 1.659785]
epoch:29 step:27843 [D loss: 0.303004, acc.: 92.97%] [G loss: 1.567686]
epoch:29 step:27844 [D loss: 0.368230, acc.: 85.16%] [G loss: 1.758974]
epoch:29 step:27845 [D loss: 0.395543, acc.: 81.25%] [G loss: 1.845025]
epoch:29 step:27846 [D loss: 1.275882, acc.: 17.97%] [G loss: 0.656735]
epoch:29 step:27847 [D loss: 0.405544, acc.: 81.25%] [G loss: 1.325673]
epoch:29 step:27848 [D loss: 0.855149, acc.: 46.09%] [G loss: 1.035455]
epoch:29 step:27849 [D loss: 0.446500, acc.: 79.69%] [G loss: 1.

epoch:29 step:27951 [D loss: 0.902155, acc.: 36.72%] [G loss: 1.108048]
epoch:29 step:27952 [D loss: 0.749869, acc.: 48.44%] [G loss: 0.865093]
epoch:29 step:27953 [D loss: 0.495237, acc.: 78.12%] [G loss: 1.420124]
epoch:29 step:27954 [D loss: 0.503426, acc.: 79.69%] [G loss: 1.239594]
epoch:29 step:27955 [D loss: 0.450241, acc.: 80.47%] [G loss: 1.311278]
epoch:29 step:27956 [D loss: 0.695983, acc.: 60.94%] [G loss: 1.264130]
epoch:29 step:27957 [D loss: 0.767227, acc.: 44.53%] [G loss: 0.942273]
epoch:29 step:27958 [D loss: 0.556600, acc.: 70.31%] [G loss: 1.049411]
epoch:29 step:27959 [D loss: 0.496718, acc.: 79.69%] [G loss: 1.130657]
epoch:29 step:27960 [D loss: 0.877007, acc.: 39.84%] [G loss: 1.071839]
epoch:29 step:27961 [D loss: 0.675426, acc.: 62.50%] [G loss: 0.925036]
epoch:29 step:27962 [D loss: 0.728630, acc.: 53.91%] [G loss: 1.034126]
epoch:29 step:27963 [D loss: 0.521548, acc.: 76.56%] [G loss: 0.843694]
epoch:29 step:27964 [D loss: 0.309049, acc.: 90.62%] [G loss: 1.

epoch:29 step:28066 [D loss: 0.334018, acc.: 90.62%] [G loss: 1.946773]
epoch:29 step:28067 [D loss: 0.320383, acc.: 89.84%] [G loss: 1.562225]
epoch:29 step:28068 [D loss: 0.572255, acc.: 70.31%] [G loss: 1.477905]
epoch:29 step:28069 [D loss: 0.195385, acc.: 98.44%] [G loss: 1.899957]
epoch:29 step:28070 [D loss: 0.202001, acc.: 98.44%] [G loss: 1.842049]
epoch:29 step:28071 [D loss: 0.154938, acc.: 98.44%] [G loss: 2.035897]
epoch:29 step:28072 [D loss: 0.109962, acc.: 100.00%] [G loss: 2.762444]
epoch:29 step:28073 [D loss: 0.125748, acc.: 100.00%] [G loss: 2.688625]
epoch:29 step:28074 [D loss: 0.221241, acc.: 98.44%] [G loss: 1.803971]
epoch:29 step:28075 [D loss: 0.373411, acc.: 84.38%] [G loss: 2.093618]
epoch:29 step:28076 [D loss: 0.360259, acc.: 85.94%] [G loss: 1.911591]
epoch:29 step:28077 [D loss: 0.665246, acc.: 60.16%] [G loss: 1.291614]
epoch:29 step:28078 [D loss: 0.432973, acc.: 82.81%] [G loss: 1.602618]
epoch:29 step:28079 [D loss: 0.547176, acc.: 70.31%] [G loss: 